# 연구목표 수립

- 최종 산출물 고려해서 설계
- 데이터 MINIST(손글씨)를 이용해서 0~9까지 손글씨 이미지를 인식하는 딥러닝 모델 구축
- 손글씨를 웹에서 canvas에 그리고 이를 판독해서 숫자로 데이터로 추출(생략)
- 딥러닝에 사용할 인공신경망은 cnn 활용
- 엔진은 Tensorflow 1.x 사용
- 데이터 훈련용
  - shape => (60000, 28*28)
- 테스트용 훈련용
  - shape => (10000, 28*28)
- **텐서플로우 활용, cnn이해 / 구현, 전체 플로우 이해**

In [ ]:
!pip install tensorflow==1.15

%tensorflow_version 1.x

import tensorflow as tf
tf.__version__

     |████████████████████████████████| 412.3 MB 22 kB/s 
     |████████████████████████████████| 50 kB 6.2 MB/s 
     |████████████████████████████████| 503 kB 50.2 MB/s 
     |████████████████████████████████| 3.8 MB 28.0 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=aedb98aa423ec2e9b62bff1c0c3c6308295806d0b0d04fa506078536a689f9f7
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Succes

'1.15.2'

# 데이터 수집

In [ ]:
mnist = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
type(mnist), len(mnist)

(tuple, 2)

In [ ]:
# 용도별로 분해
mnist_train, mnist_test = mnist

In [ ]:
type(mnist_train), len(mnist_train)

(tuple, 2)

In [ ]:
mnist_train[0].shape, mnist_test[1].shape
# 이미지 원본데이터는 3차원 
# 이미지 분류 정답 1차원

((60000, 28, 28), (10000,))

In [ ]:
# 이미지 원본 데이터
mnist_train[0][0].shape, mnist_test[0][0]
# 1 픽셀을 표현하는 값은 0(0x00) ~ 255(0xFF) => 255개의 수치로 표현

In [ ]:
# 값이 0 ~ 255까지면 값의 편차가 비교적 크다 => 정규화 처리(0~1 사이로)
# 최대값 255로 나누면 0~1 사이로 모든 값이 정규화된다
X_train = mnist_train[0] / 255
X_train[0][0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
# 이미 처리된 데이터 활용
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
from numpy.core.numeric import ones
# 정답이 0과 1로 표현된다 : one_hot=True
mnist = input_data.read_data_sets('./data/mnist', one_hot = True)

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


In [ ]:
mnist.train.images.shape, mnist.validation.images.shape, mnist.test.images.shape

((55000, 784), (5000, 784), (10000, 784))

In [ ]:
mnist.train.images[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

- 데이터는 55000개, 5000, 10000개 준비
- 이미지의 최종 크기는 27*27 이고 데이터 (55000, 784) 준비
- 각 픽셀은 0~1 사이 값으로 설정되어 있음


# 데이터 준비 (생략)

# 데이터 분석 (생략)

# 모델 구축 (딥러닝-CNN)

- 목표
  - CNN을 기반으로 한 인공신경망 mnist에서 제공하는 데이터(손글씨이미지)를 잘 분류해 내도록 학습 -> 최적화 -> 가장 높은 성능을 내는 W, b를 찾아내는 과정
  - 이를 구현하기 위해서 텐서플로우 1.x 엔진을 활용한다


## 환경변수 설정
- 딥러닝 수행 시 필요한 상수값 정의
- 학습시 필요한 통제 환경을 정의
 

In [ ]:
# 전체 피쳐 수, 전체 픽셀 수 
PIXEL = mnist.train.images.shape[1]
PIXEL

784

In [ ]:
import numpy as np

In [ ]:
# 이미지의 세로, 가로 크기 => 현재 이미지는 정사각형
PIXEL_H = int(np.sqrt(PIXEL))
PIXEL_W = PIXEL_H

PIXEL_H, PIXEL_W

(28, 28)

In [ ]:
# 정답의 중복되지 않는 종류의 총 개수
LABEL_NUM = mnist.train.labels.shape[-1]
LABEL_NUM

10

In [ ]:
mnist.train.labels

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

## CNN을 이용한 인공신경망 구축



```
CNN Network

- 입력층     : Input layer
- 은닉층     : hidden layer
  - 합성곱층 : convolution layer
  - 풀링층   : pooling layer
  - 합성곱층 : convolution layer
  - 풀링층   : pooling layer
  - 전결합층 : Fully connected layer
  - 드롭아웃층(과적합 방지) : dropout layer 
- 출력층 : output layer

55000개의 데이터를 1세대 학습에서 모두 사용
```




### PART1 데이터 플로우 그래프 구성

#### 입력층

- 변수명 : x
- 목적 : 손글씨 이미지 데이터가 주입된다(외부에서 데이터를 넣는다) -> 학습하기 위해서 
- 타입 : placehoder
- shape :
  - 훈련데이터 (55000, 784)
  - 검증데이터 (5000, 784)
  - 테스트데이터 (10000, 784)
  - => (?, 784) -> 가변형 placeholder 구성
- 주입되는 데이터 타입 : float

In [ ]:
x = tf.placeholder(tf.float32, shape = (None, PIXEL), name = 'x')
x

<tf.Tensor 'x:0' shape=(?, 784) dtype=float32>

#### 은닉층 > 합성곱층 1F

- 목적
  - 이미지 상의 공간 / 인접 등 정보(특징)을 추출하는 단계
  - 이미지 공간 상을 필터(커널)가 이동(슬라이딩, 스트라이드 양만큼)하면서 연산(특징 추출), 이 값을 feature map(특징지도)를 만드는 층
  - 이미지늘 이해하기 위해서 
  - 조건
    - 정확도를 높이기 위해서는 체크하는 이미지 외에 다른 이미지가 없어야 좋다
    - MNIST의 뒷배경은 검정색
    - **일반적인 이미지**로 대비해 보면 **뒷 배경에 잡음이 많다** -> 바운드박스 등등 **라벨링** 이후 추출과정(opencv 활용)

- 입력
  - 이전 층의 출력
- 변수 
  - act_conv_1f
- 구성요소 
  - k : Kernel(커널, 필터)
    - 커널의 크기 정보 필요, 2D
    - (h, w)
    - 통상 정방형 커널 사용, 세로 & 가로 사이즈 동일
  - W : Weight(가중치)
    - 커널이 들고 있는 값
    - 커널의 공용 파라미터
  - b : bias(편향)
    - 전체적인 입력대비 출력값을 조절하는 역할
  - s : stride
    - 커널을 이동시키는 양 (가로, 세로)
    - 왼 -> 오른쪽 , 위 -> 아래로 이동
  - p : padding
    - 보정, 커널이 이동하는 중 경계선에서 보정할 것인지 다음으로 이동할 것인지 처리하는 기준
    - SAME(동일크기), VALID(유효범위만, 일반적으로 축소됨)
- 출력
  - feature map or activation map
- shape 
  - x : (None, 784)
  - W(or k) : (5, 5) <- 설정
  - b : (32,) <- 설정
  - 1장의 이미지를 넣어서 => 32개의 이미지로 추출


#### 은닉층 > 풀링층 1F

#### 은닉층 > 합성곱층 2F

#### 은닉층 > 풀링층 2F

#### 은닉층 > 전결합층

#### 은닉층 > 드롭아웃층

#### 출력층

### PART2 데이터 주입 및 학습 및 예측수행

- 데이터 주입 형태
  - feed_dict = {x : (?, 784)}

# 시스템 통합 / 산출물